# Milestone 07: 待辦事項管理系統 - 完整參考實作

## 📖 解答說明

本解答提供待辦事項管理系統的**完整參考實作**,整合 Ch23-26 的所有核心技術。

### 🎯 學習重點

| 技術 | 應用 | 章節 |
|:-----|:-----|:-----|
| **JSON 序列化** | 資料持久化儲存 | Ch24 |
| **pathlib** | 跨平台路徑管理 | Ch26 |
| **datetime** | 時間處理與格式化 | Ch24 |
| **UUID** | 唯一識別碼生成 | - |
| **CSV 匯出** | 報表生成與資料分析 | Ch25 |
| **檔案備份** | 資料安全與錯誤恢復 | Ch23 |
| **例外處理** | 健全的錯誤管理 | Ch20-21 |

### 🏗️ 架構設計

```
Task (資料模型)
  ↓
TodoManager (業務邏輯)
  ↓
TodoApp (使用者介面)
```

### 📂 檔案結構

```
data/
├── todos.json          # 主要資料檔案
├── todos_backup.json   # 自動備份
└── exports/
    └── tasks_export.csv
```

### 💡 設計亮點

1. **資料持久化**: 使用 JSON 儲存,包含自動備份機制
2. **路徑管理**: pathlib 確保跨平台相容性
3. **時間處理**: ISO 8601 格式,易於序列化與解析
4. **錯誤恢復**: 檔案損毀時自動從備份恢復
5. **搜尋過濾**: 不區分大小寫,支援多條件查詢
6. **資料匯出**: CSV 格式便於 Excel 分析
7. **統計報表**: 任務狀態統計與完成率計算

---

## 📝 實作說明

### 完成度：100% （基本需求 70分 + 進階需求 30分）

**基本需求 (70分)**:
- ✅ Task 資料結構 (10分)
- ✅ 檔案持久化 (15分)
- ✅ CRUD 操作 (25分)
- ✅ 搜尋與過濾 (10分)
- ✅ CLI 介面 (10分)

**進階需求 (30分)**:
- ✅ 截止日期管理 (8分)
- ✅ CSV 匯出 (8分)
- ✅ 統計報表 (5分)
- ✅ 標籤系統 (6分)
- ✅ 逾期檢查 (3分)

**程式碼品質**:
- ✅ 完整的 docstring 文檔
- ✅ 健全的例外處理
- ✅ 清晰的模組化設計
- ✅ 友善的使用者體驗

---

## Part I: 匯入模組與環境設定

In [ ]:
"""待辦事項管理系統 - 必要模組匯入"""

# 標準函式庫
import json                    # JSON 序列化與反序列化
import csv                     # CSV 檔案處理
import uuid                    # UUID 唯一識別碼生成
import shutil                  # 檔案操作（備份）
from pathlib import Path       # 跨平台路徑管理
from datetime import datetime, timedelta  # 時間處理
from typing import List, Dict, Optional   # 型別提示

print("✓ 所有模組載入成功")
print(f"✓ 當前工作目錄: {Path.cwd()}")
print(f"✓ Python 版本: 3.x (需要 3.7+)")

## Part II: Task 資料模型（Data Model）

### 設計考量

1. **唯一識別**: 使用 UUID4 生成唯一 ID
2. **時間處理**: ISO 8601 格式（YYYY-MM-DDTHH:MM:SS）
3. **資料驗證**: 優先級與狀態必須為預定義值
4. **序列化支援**: to_dict() 與 from_dict() 便於 JSON 轉換
5. **逾期檢查**: is_overdue() 方法檢查截止日期

In [ ]:
class Task:
    """待辦任務資料模型
    
    Attributes:
        id (str): 唯一識別碼（UUID4 格式）
        title (str): 任務標題（1-100字元）
        description (str): 任務描述
        priority (str): 優先級 (High/Medium/Low)
        status (str): 狀態 (Pending/InProgress/Completed)
        created_date (datetime): 建立時間
        due_date (datetime): 截止時間（可為 None）
        tags (List[str]): 標籤列表
    """
    
    # 類別常數：定義有效的優先級與狀態
    VALID_PRIORITIES = ["High", "Medium", "Low"]
    VALID_STATUSES = ["Pending", "InProgress", "Completed"]
    
    def __init__(self, 
                 title: str,
                 description: str = "",
                 priority: str = "Medium",
                 due_date: Optional[str] = None,
                 tags: Optional[List[str]] = None,
                 task_id: Optional[str] = None,
                 status: str = "Pending",
                 created_date: Optional[str] = None):
        """初始化任務物件
        
        Args:
            title: 任務標題（必填）
            description: 任務描述
            priority: 優先級 (High/Medium/Low)
            due_date: 截止日期（ISO格式字串或 None）
            tags: 標籤列表
            task_id: 任務 ID（若未提供則自動生成）
            status: 任務狀態
            created_date: 建立時間（ISO格式字串或 None）
        
        Raises:
            ValueError: 標題為空、過長或優先級無效
        """
        # 1. 驗證標題
        if not title or not title.strip():
            raise ValueError("任務標題不能為空")
        if len(title) > 100:
            raise ValueError("任務標題長度不能超過 100 字元")
        
        # 2. 驗證優先級
        if priority not in self.VALID_PRIORITIES:
            raise ValueError(f"無效的優先級：{priority}。必須為 {self.VALID_PRIORITIES} 之一")
        
        # 3. 驗證狀態
        if status not in self.VALID_STATUSES:
            raise ValueError(f"無效的狀態：{status}。必須為 {self.VALID_STATUSES} 之一")
        
        # 4. 設定屬性
        self.id = task_id if task_id else str(uuid.uuid4())
        self.title = title.strip()
        self.description = description.strip()
        self.priority = priority
        self.status = status
        self.tags = tags if tags else []
        
        # 5. 處理時間（ISO 8601 格式）
        if created_date:
            self.created_date = datetime.fromisoformat(created_date)
        else:
            self.created_date = datetime.now()
        
        if due_date:
            try:
                self.due_date = datetime.fromisoformat(due_date)
            except ValueError:
                # 支援簡化格式 YYYY-MM-DD
                self.due_date = datetime.strptime(due_date, "%Y-%m-%d")
        else:
            self.due_date = None
    
    def to_dict(self) -> Dict:
        """將 Task 物件轉換為字典（用於 JSON 序列化）
        
        Returns:
            Dict: 包含所有任務屬性的字典
        """
        return {
            "id": self.id,
            "title": self.title,
            "description": self.description,
            "priority": self.priority,
            "status": self.status,
            "created_date": self.created_date.isoformat(),  # datetime → ISO 字串
            "due_date": self.due_date.isoformat() if self.due_date else None,
            "tags": self.tags
        }
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'Task':
        """從字典建立 Task 物件（用於 JSON 反序列化）
        
        Args:
            data: 包含任務資料的字典
        
        Returns:
            Task: 新建立的任務物件
        """
        return cls(
            title=data["title"],
            description=data.get("description", ""),
            priority=data.get("priority", "Medium"),
            due_date=data.get("due_date"),
            tags=data.get("tags", []),
            task_id=data.get("id"),
            status=data.get("status", "Pending"),
            created_date=data.get("created_date")
        )
    
    def is_overdue(self) -> bool:
        """檢查任務是否逾期
        
        Returns:
            bool: 是否逾期（無截止日期或已完成則返回 False）
        """
        if not self.due_date or self.status == "Completed":
            return False
        return datetime.now() > self.due_date
    
    def days_until_due(self) -> Optional[int]:
        """計算距離截止日期的天數
        
        Returns:
            int: 天數（負數表示已逾期）,若無截止日期則返回 None
        """
        if not self.due_date:
            return None
        delta = self.due_date - datetime.now()
        return delta.days
    
    def __str__(self) -> str:
        """回傳友善的字串表示
        
        格式: [優先級] 標題 (狀態) - 截止日期 [逾期標示] #標籤
        範例: [High] 完成 Python 專案 (InProgress) - 2025-10-15 ⚠️ #work #urgent
        """
        # 基本資訊
        result = f"[{self.priority}] {self.title} ({self.status})"
        
        # 截止日期
        if self.due_date:
            due_str = self.due_date.strftime("%Y-%m-%d %H:%M")
            result += f" - 截止: {due_str}"
            
            # 逾期標示
            if self.is_overdue():
                result += " ⚠️ 已逾期"
            else:
                days = self.days_until_due()
                if days is not None and days <= 3:
                    result += f" ⏰ 剩 {days} 天"
        
        # 標籤
        if self.tags:
            tags_str = " ".join(f"#{tag}" for tag in self.tags)
            result += f" {tags_str}"
        
        return result
    
    def __repr__(self) -> str:
        """回傳物件的詳細表示（用於除錯）"""
        return f"Task(id={self.id[:8]}, title='{self.title}', status={self.status})"
    
    def __eq__(self, other) -> bool:
        """比較兩個 Task 是否相等（根據 ID）"""
        if not isinstance(other, Task):
            return False
        return self.id == other.id


# 測試 Task 類別
print("\n=== Task 類別測試 ===")

# 建立測試任務
task1 = Task(
    title="完成 Python 專案",
    description="實作待辦事項管理系統",
    priority="High",
    due_date="2025-10-15",
    tags=["work", "programming"]
)

print(f"✓ 任務建立成功: {task1}")
print(f"✓ 任務 ID: {task1.id}")
print(f"✓ 是否逾期: {task1.is_overdue()}")
print(f"✓ 剩餘天數: {task1.days_until_due()}")

# 測試序列化
task_dict = task1.to_dict()
print(f"\n✓ 轉換為字典: {json.dumps(task_dict, ensure_ascii=False, indent=2)}")

# 測試反序列化
task2 = Task.from_dict(task_dict)
print(f"\n✓ 從字典還原: {task2}")
print(f"✓ 物件相等性: {task1 == task2}")

## Part III: TodoManager 類別 - 檔案持久化

### 設計重點

1. **自動備份**: 每次儲存前先備份舊檔案
2. **錯誤恢復**: 檔案損毀時自動從備份恢復
3. **路徑管理**: 使用 pathlib 確保跨平台相容
4. **目錄建立**: 自動建立 data 和 exports 目錄

In [ ]:
class TodoManager:
    """待辦事項管理器 - 檔案持久化與業務邏輯
    
    Attributes:
        tasks (List[Task]): 所有任務列表
        data_dir (Path): 資料目錄路徑
        data_file (Path): 主要資料檔案路徑
        backup_file (Path): 備份檔案路徑
        export_dir (Path): 匯出目錄路徑
    """
    
    def __init__(self, data_dir: str = "data"):
        """初始化 TodoManager
        
        Args:
            data_dir: 資料目錄名稱（預設為 'data'）
        """
        # 1. 設定路徑（使用 pathlib）
        self.data_dir = Path(data_dir)
        self.data_file = self.data_dir / "todos.json"
        self.backup_file = self.data_dir / "todos_backup.json"
        self.export_dir = self.data_dir / "exports"
        
        # 2. 建立目錄（如果不存在）
        self.data_dir.mkdir(exist_ok=True)
        self.export_dir.mkdir(exist_ok=True)
        
        # 3. 初始化任務列表
        self.tasks: List[Task] = []
        
        print(f"✓ TodoManager 初始化完成")
        print(f"  - 資料目錄: {self.data_dir.absolute()}")
        print(f"  - 主檔案: {self.data_file.name}")
        print(f"  - 備份檔案: {self.backup_file.name}")
    
    def save_to_file(self) -> bool:
        """儲存所有任務至 JSON 檔案
        
        步驟：
        1. 將所有 Task 物件轉換為字典列表
        2. 建立 metadata（最後修改時間、版本、任務數量）
        3. 備份現有檔案（如果存在）
        4. 寫入新資料至檔案
        
        Returns:
            bool: 儲存是否成功
        
        Raises:
            IOError: 檔案寫入失敗
            PermissionError: 沒有寫入權限
        """
        try:
            # 1. 準備資料
            data = {
                "tasks": [task.to_dict() for task in self.tasks],
                "metadata": {
                    "last_modified": datetime.now().isoformat(),
                    "version": "1.0",
                    "total_tasks": len(self.tasks)
                }
            }
            
            # 2. 建立備份（如果主檔案存在）
            if self.data_file.exists():
                shutil.copy(self.data_file, self.backup_file)
            
            # 3. 寫入檔案
            with open(self.data_file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            return True
        
        except PermissionError:
            print(f"❌ 錯誤：沒有權限寫入檔案 {self.data_file}")
            return False
        except IOError as e:
            print(f"❌ 檔案寫入失敗：{e}")
            return False
        except Exception as e:
            print(f"❌ 儲存時發生未預期的錯誤：{e}")
            return False
    
    def load_from_file(self) -> bool:
        """從 JSON 檔案載入任務
        
        步驟：
        1. 檢查檔案是否存在
        2. 讀取並解析 JSON
        3. 將字典資料轉換為 Task 物件
        4. 處理檔案損毀情況（嘗試從備份恢復）
        
        Returns:
            bool: 載入是否成功
        """
        try:
            # 1. 檔案不存在時建立空列表
            if not self.data_file.exists():
                print("ℹ️  資料檔案不存在,建立新的任務列表")
                self.tasks = []
                return True
            
            # 2. 讀取檔案
            with open(self.data_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # 3. 轉換為 Task 物件
            self.tasks = [
                Task.from_dict(task_data)
                for task_data in data.get("tasks", [])
            ]
            
            print(f"✓ 成功載入 {len(self.tasks)} 個任務")
            return True
        
        except json.JSONDecodeError:
            # JSON 格式錯誤,嘗試從備份恢復
            print("⚠️  主檔案損毀,嘗試從備份恢復...")
            return self._restore_from_backup()
        
        except FileNotFoundError:
            print("ℹ️  資料檔案不存在,建立新的任務列表")
            self.tasks = []
            return True
        
        except Exception as e:
            print(f"❌ 載入失敗：{e}")
            return False
    
    def _restore_from_backup(self) -> bool:
        """從備份檔案恢復資料（內部方法）
        
        Returns:
            bool: 恢復是否成功
        """
        try:
            if not self.backup_file.exists():
                print("❌ 備份檔案不存在,無法恢復")
                self.tasks = []
                return False
            
            # 從備份讀取
            with open(self.backup_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            self.tasks = [
                Task.from_dict(task_data)
                for task_data in data.get("tasks", [])
            ]
            
            # 將備份複製回主檔案
            shutil.copy(self.backup_file, self.data_file)
            
            print(f"✓ 成功從備份恢復 {len(self.tasks)} 個任務")
            return True
        
        except Exception as e:
            print(f"❌ 從備份恢復失敗：{e}")
            self.tasks = []
            return False
    
    def create_backup(self) -> bool:
        """手動建立備份
        
        Returns:
            bool: 備份是否成功
        """
        try:
            if not self.data_file.exists():
                print("⚠️  主檔案不存在,無法建立備份")
                return False
            
            shutil.copy(self.data_file, self.backup_file)
            print(f"✓ 備份已建立: {self.backup_file}")
            return True
        except Exception as e:
            print(f"❌ 建立備份失敗：{e}")
            return False


# 測試檔案持久化
print("\n=== 檔案持久化測試 ===")

manager = TodoManager("data")

# 建立測試任務
task1 = Task("測試任務 1", priority="High", tags=["test"])
task2 = Task("測試任務 2", priority="Low", due_date="2025-12-31")
manager.tasks = [task1, task2]

# 儲存測試
print("\n1. 測試儲存功能...")
if manager.save_to_file():
    print("✓ 儲存成功")

# 讀取測試
print("\n2. 測試讀取功能...")
manager2 = TodoManager("data")
if manager2.load_from_file():
    print(f"✓ 讀取成功,共 {len(manager2.tasks)} 個任務")
    for task in manager2.tasks:
        print(f"  - {task}")

# 備份測試
print("\n3. 測試備份功能...")
if manager2.create_backup():
    print("✓ 備份測試完成")

## Part IV: TodoManager 類別 - CRUD 操作

### 功能說明

- **Create**: add_task() - 新增任務
- **Read**: list_tasks() - 列出任務（支援過濾與排序）
- **Update**: update_task() - 更新任務
- **Delete**: delete_task() - 刪除任務

In [ ]:
class TodoManager(TodoManager):  # 擴展原有類別
    """TodoManager - CRUD 操作擴展"""
    
    def add_task(self,
                 title: str,
                 description: str = "",
                 priority: str = "Medium",
                 due_date: Optional[str] = None,
                 tags: Optional[List[str]] = None) -> Optional[Task]:
        """新增待辦任務
        
        Args:
            title: 任務標題（必填,1-100字元）
            description: 任務描述
            priority: 優先級（High/Medium/Low）
            due_date: 截止日期（YYYY-MM-DD 或 ISO格式）
            tags: 標籤列表
        
        Returns:
            Task: 新建立的任務物件,失敗則返回 None
        """
        try:
            # 1. 建立 Task 物件（內建驗證）
            task = Task(title, description, priority, due_date, tags)
            
            # 2. 加入任務清單
            self.tasks.append(task)
            
            # 3. 儲存至檔案
            self.save_to_file()
            
            print(f"✓ 任務已新增：{task.title}")
            return task
        
        except ValueError as e:
            print(f"❌ 新增失敗：{e}")
            return None
    
    def list_tasks(self,
                   filter_by: Optional[str] = None,
                   sort_by: str = "created") -> List[Task]:
        """列出所有任務,支援過濾與排序
        
        Args:
            filter_by: 過濾條件
                - 狀態: 'pending', 'inprogress', 'completed'
                - 優先級: 'high', 'medium', 'low'
                - 特殊: 'overdue' (逾期), 'upcoming' (即將到期)
            sort_by: 排序依據
                - 'created': 建立時間（預設）
                - 'due_date': 截止日期
                - 'priority': 優先級（High → Low）
        
        Returns:
            List[Task]: 過濾並排序後的任務列表
        """
        # 1. 過濾
        filtered_tasks = self.tasks
        
        if filter_by:
            filter_lower = filter_by.lower()
            
            # 狀態過濾
            if filter_lower in ["pending", "inprogress", "completed"]:
                filtered_tasks = [
                    t for t in self.tasks
                    if t.status.lower() == filter_lower
                ]
            # 優先級過濾
            elif filter_lower in ["high", "medium", "low"]:
                filtered_tasks = [
                    t for t in self.tasks
                    if t.priority.lower() == filter_lower
                ]
            # 逾期過濾
            elif filter_lower == "overdue":
                filtered_tasks = [t for t in self.tasks if t.is_overdue()]
            # 即將到期過濾（3天內）
            elif filter_lower == "upcoming":
                filtered_tasks = [
                    t for t in self.tasks
                    if t.due_date and 0 <= t.days_until_due() <= 3
                ]
        
        # 2. 排序
        if sort_by == "priority":
            priority_order = {"High": 0, "Medium": 1, "Low": 2}
            filtered_tasks.sort(key=lambda t: priority_order[t.priority])
        elif sort_by == "due_date":
            # 有截止日期的排前面,按日期排序
            filtered_tasks.sort(
                key=lambda t: t.due_date if t.due_date else datetime.max
            )
        else:  # 預設按建立時間
            filtered_tasks.sort(key=lambda t: t.created_date)
        
        # 3. 顯示
        if not filtered_tasks:
            print("\n目前沒有符合條件的任務")
        else:
            print(f"\n共 {len(filtered_tasks)} 個任務：")
            print("-" * 80)
            for i, task in enumerate(filtered_tasks, 1):
                print(f"{i:2d}. {task}")
                if task.description:
                    print(f"    說明: {task.description}")
            print("-" * 80)
        
        return filtered_tasks
    
    def _find_task_by_id(self, task_id: str) -> Optional[Task]:
        """根據 ID 尋找任務（內部方法）
        
        Args:
            task_id: 任務 ID（可為完整 UUID 或前綴）
        
        Returns:
            Task: 找到的任務,否則返回 None
        """
        for task in self.tasks:
            if task.id == task_id or task.id.startswith(task_id):
                return task
        return None
    
    def update_task(self, task_id: str, **kwargs) -> bool:
        """更新任務的指定欄位
        
        Args:
            task_id: 任務 ID
            **kwargs: 要更新的欄位
                - title: 標題
                - description: 描述
                - priority: 優先級
                - status: 狀態
                - due_date: 截止日期
                - tags: 標籤列表
        
        Returns:
            bool: 更新是否成功
        """
        # 1. 找到任務
        task = self._find_task_by_id(task_id)
        if not task:
            print(f"❌ 找不到任務 ID: {task_id}")
            return False
        
        # 2. 更新欄位
        try:
            for key, value in kwargs.items():
                if key == "due_date" and value:
                    # 特殊處理截止日期
                    task.due_date = datetime.fromisoformat(value)
                elif key == "priority" and value not in Task.VALID_PRIORITIES:
                    raise ValueError(f"無效的優先級：{value}")
                elif key == "status" and value not in Task.VALID_STATUSES:
                    raise ValueError(f"無效的狀態：{value}")
                elif hasattr(task, key):
                    setattr(task, key, value)
            
            # 3. 儲存變更
            self.save_to_file()
            print(f"✓ 任務已更新：{task.title}")
            return True
        
        except ValueError as e:
            print(f"❌ 更新失敗：{e}")
            return False
    
    def delete_task(self, task_id: str, confirm: bool = True) -> bool:
        """刪除指定任務
        
        Args:
            task_id: 任務 ID
            confirm: 是否需要確認（預設 True）
        
        Returns:
            bool: 刪除是否成功
        """
        # 1. 找到任務
        task = self._find_task_by_id(task_id)
        if not task:
            print(f"❌ 找不到任務 ID: {task_id}")
            return False
        
        # 2. 確認刪除
        if confirm:
            response = input(f"\n確定要刪除任務「{task.title}」嗎？(y/n): ")
            if response.lower() != 'y':
                print("已取消刪除")
                return False
        
        # 3. 移除任務並儲存
        self.tasks.remove(task)
        self.save_to_file()
        print(f"✓ 任務已刪除：{task.title}")
        return True
    
    def mark_completed(self, task_id: str) -> bool:
        """標記任務為已完成
        
        Args:
            task_id: 任務 ID
        
        Returns:
            bool: 操作是否成功
        """
        return self.update_task(task_id, status="Completed")
    
    def mark_in_progress(self, task_id: str) -> bool:
        """標記任務為進行中
        
        Args:
            task_id: 任務 ID
        
        Returns:
            bool: 操作是否成功
        """
        return self.update_task(task_id, status="InProgress")


# 測試 CRUD 操作
print("\n=== CRUD 操作測試 ===")

manager = TodoManager("data")
manager.load_from_file()

# 新增任務
print("\n1. 測試新增任務...")
task1 = manager.add_task(
    "準備期中考",
    "複習 Python 基礎語法",
    "High",
    "2025-10-20",
    ["study", "exam"]
)

task2 = manager.add_task(
    "撰寫報告",
    "完成資料結構課程報告",
    "Medium",
    "2025-10-25"
)

# 列出任務
print("\n2. 測試列出所有任務...")
manager.list_tasks()

# 過濾任務
print("\n3. 測試過濾功能（高優先級）...")
manager.list_tasks(filter_by="High")

# 更新任務
if task1:
    print("\n4. 測試更新任務...")
    manager.update_task(task1.id[:8], status="InProgress")

# 標記完成
if task2:
    print("\n5. 測試標記完成...")
    manager.mark_completed(task2.id[:8])

## Part V: TodoManager 類別 - 搜尋與過濾

### 功能說明

- **關鍵字搜尋**: 不區分大小寫,搜尋標題與描述
- **條件過濾**: 優先級、狀態、標籤、逾期
- **標籤管理**: 列出所有標籤、標籤統計

In [ ]:
class TodoManager(TodoManager):  # 繼續擴展
    """TodoManager - 搜尋與過濾功能"""
    
    def search_tasks(self, keyword: str) -> List[Task]:
        """搜尋標題或描述中包含關鍵字的任務
        
        Args:
            keyword: 搜尋關鍵字（不區分大小寫）
        
        Returns:
            List[Task]: 符合條件的任務列表
        """
        if not keyword:
            print("⚠️  請輸入搜尋關鍵字")
            return []
        
        keyword_lower = keyword.lower()
        
        # 搜尋標題、描述、標籤
        results = [
            task for task in self.tasks
            if keyword_lower in task.title.lower()
            or keyword_lower in task.description.lower()
            or any(keyword_lower in tag.lower() for tag in task.tags)
        ]
        
        # 顯示結果
        print(f"\n搜尋「{keyword}」找到 {len(results)} 個任務：")
        if results:
            print("-" * 80)
            for i, task in enumerate(results, 1):
                print(f"{i}. {task}")
            print("-" * 80)
        else:
            print("沒有找到符合的任務")
        
        return results
    
    def filter_by_priority(self, priority: str) -> List[Task]:
        """依照優先級過濾任務
        
        Args:
            priority: 優先級（High/Medium/Low）
        
        Returns:
            List[Task]: 符合條件的任務
        """
        return [task for task in self.tasks if task.priority == priority]
    
    def filter_by_status(self, status: str) -> List[Task]:
        """依照狀態過濾任務
        
        Args:
            status: 狀態（Pending/InProgress/Completed）
        
        Returns:
            List[Task]: 符合條件的任務
        """
        return [task for task in self.tasks if task.status == status]
    
    def filter_by_tag(self, tag: str) -> List[Task]:
        """依照標籤過濾任務
        
        Args:
            tag: 標籤名稱
        
        Returns:
            List[Task]: 包含該標籤的任務
        """
        results = [task for task in self.tasks if tag in task.tags]
        print(f"\n標籤「{tag}」共有 {len(results)} 個任務")
        return results
    
    def get_overdue_tasks(self) -> List[Task]:
        """取得所有逾期任務
        
        Returns:
            List[Task]: 逾期的任務列表
        """
        overdue = [task for task in self.tasks if task.is_overdue()]
        
        print(f"\n⚠️  有 {len(overdue)} 個任務已逾期：")
        if overdue:
            for task in overdue:
                print(f"  - {task}")
        
        return overdue
    
    def get_upcoming_tasks(self, days: int = 3) -> List[Task]:
        """取得未來 N 天內到期的任務
        
        Args:
            days: 天數（預設 3 天）
        
        Returns:
            List[Task]: 即將到期的任務
        """
        upcoming = [
            task for task in self.tasks
            if task.due_date and task.status != "Completed"
            and 0 <= task.days_until_due() <= days
        ]
        
        print(f"\n⏰ 未來 {days} 天內有 {len(upcoming)} 個任務即將到期：")
        if upcoming:
            for task in upcoming:
                print(f"  - {task}")
        
        return upcoming
    
    def get_all_tags(self) -> List[str]:
        """取得所有使用中的標籤
        
        Returns:
            List[str]: 標籤列表（不重複）
        """
        all_tags = set()
        for task in self.tasks:
            all_tags.update(task.tags)
        
        return sorted(all_tags)
    
    def get_tag_statistics(self) -> Dict[str, int]:
        """取得標籤統計
        
        Returns:
            Dict[str, int]: 標籤與使用次數的字典
        """
        tag_counts = {}
        for task in self.tasks:
            for tag in task.tags:
                tag_counts[tag] = tag_counts.get(tag, 0) + 1
        
        # 按使用次數排序
        return dict(sorted(tag_counts.items(), key=lambda x: x[1], reverse=True))


# 測試搜尋與過濾
print("\n=== 搜尋與過濾測試 ===")

manager = TodoManager("data")
manager.load_from_file()

# 新增測試資料
manager.add_task("Python 作業", "完成第三章習題", "High", "2025-10-10", ["study", "programming"])
manager.add_task("Java 報告", "撰寫物件導向報告", "Medium", "2025-10-15", ["study"])
manager.add_task("買菜", "準備晚餐食材", "Low", tags=["life"])

# 關鍵字搜尋
print("\n1. 測試關鍵字搜尋...")
manager.search_tasks("報告")

# 標籤過濾
print("\n2. 測試標籤過濾...")
manager.filter_by_tag("study")

# 逾期檢查
print("\n3. 測試逾期檢查...")
manager.get_overdue_tasks()

# 即將到期
print("\n4. 測試即將到期提醒...")
manager.get_upcoming_tasks(days=30)

# 標籤統計
print("\n5. 測試標籤統計...")
tag_stats = manager.get_tag_statistics()
print(f"\n標籤使用統計：")
for tag, count in tag_stats.items():
    print(f"  #{tag}: {count} 次")

## Part VI: TodoManager 類別 - CSV 匯出與統計

### 功能說明

- **CSV 匯出**: 匯出所有任務為 CSV 格式
- **統計報表**: 任務數量、完成率、優先級分布
- **資料分析**: 提供可用於 Excel 分析的資料

In [ ]:
class TodoManager(TodoManager):  # 繼續擴展
    """TodoManager - CSV 匯出與統計功能"""
    
    def export_to_csv(self, filename: str = "tasks_export.csv") -> bool:
        """匯出任務清單為 CSV 格式
        
        Args:
            filename: 匯出檔案名稱（預設 'tasks_export.csv'）
        
        Returns:
            bool: 匯出是否成功
        """
        try:
            # 1. 準備檔案路徑
            export_path = self.export_dir / filename
            
            # 2. 定義 CSV 欄位
            fieldnames = [
                "ID", "標題", "描述", "優先級", "狀態",
                "建立時間", "截止時間", "是否逾期", "剩餘天數", "標籤"
            ]
            
            # 3. 寫入 CSV
            with open(export_path, 'w', newline='', encoding='utf-8-sig') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                
                for task in self.tasks:
                    writer.writerow({
                        "ID": task.id[:8],
                        "標題": task.title,
                        "描述": task.description,
                        "優先級": task.priority,
                        "狀態": task.status,
                        "建立時間": task.created_date.strftime("%Y-%m-%d %H:%M"),
                        "截止時間": task.due_date.strftime("%Y-%m-%d %H:%M") if task.due_date else "",
                        "是否逾期": "是" if task.is_overdue() else "否",
                        "剩餘天數": task.days_until_due() if task.due_date else "",
                        "標籤": ", ".join(task.tags)
                    })
            
            print(f"✓ 已匯出 {len(self.tasks)} 個任務至: {export_path}")
            return True
        
        except IOError as e:
            print(f"❌ CSV 匯出失敗：{e}")
            return False
    
    def get_statistics(self) -> Dict:
        """取得任務統計資訊
        
        Returns:
            Dict: 包含各項統計數據的字典
        """
        total = len(self.tasks)
        
        if total == 0:
            return {
                "total": 0,
                "completed": 0,
                "in_progress": 0,
                "pending": 0,
                "overdue": 0,
                "completion_rate": 0.0
            }
        
        # 狀態統計
        completed = len([t for t in self.tasks if t.status == "Completed"])
        in_progress = len([t for t in self.tasks if t.status == "InProgress"])
        pending = len([t for t in self.tasks if t.status == "Pending"])
        overdue = len([t for t in self.tasks if t.is_overdue()])
        
        # 完成率
        completion_rate = completed / total if total > 0 else 0
        
        # 優先級統計
        high = len([t for t in self.tasks if t.priority == "High"])
        medium = len([t for t in self.tasks if t.priority == "Medium"])
        low = len([t for t in self.tasks if t.priority == "Low"])
        
        return {
            "total": total,
            "completed": completed,
            "in_progress": in_progress,
            "pending": pending,
            "overdue": overdue,
            "completion_rate": completion_rate,
            "high_priority": high,
            "medium_priority": medium,
            "low_priority": low
        }
    
    def display_statistics(self):
        """顯示統計資訊（格式化輸出）"""
        stats = self.get_statistics()
        
        print("\n" + "="*50)
        print("📊 任務統計報表")
        print("="*50)
        
        # 總覽
        print(f"\n【總覽】")
        print(f"  總任務數: {stats['total']} 個")
        print(f"  完成率: {stats['completion_rate']:.1%}")
        
        # 狀態分布
        print(f"\n【狀態分布】")
        print(f"  ✅ 已完成: {stats['completed']} 個 ({stats['completed']/stats['total']*100:.1f}%)" if stats['total'] > 0 else "  ✅ 已完成: 0 個")
        print(f"  🔄 進行中: {stats['in_progress']} 個 ({stats['in_progress']/stats['total']*100:.1f}%)" if stats['total'] > 0 else "  🔄 進行中: 0 個")
        print(f"  ⏸️  待處理: {stats['pending']} 個 ({stats['pending']/stats['total']*100:.1f}%)" if stats['total'] > 0 else "  ⏸️  待處理: 0 個")
        print(f"  ⚠️  已逾期: {stats['overdue']} 個")
        
        # 優先級分布
        print(f"\n【優先級分布】")
        print(f"  🔴 高優先級: {stats['high_priority']} 個")
        print(f"  🟡 中優先級: {stats['medium_priority']} 個")
        print(f"  🟢 低優先級: {stats['low_priority']} 個")
        
        print("="*50)
    
    def export_statistics_csv(self, filename: str = "statistics.csv") -> bool:
        """匯出統計報表為 CSV
        
        Args:
            filename: 匯出檔案名稱
        
        Returns:
            bool: 匯出是否成功
        """
        try:
            stats = self.get_statistics()
            export_path = self.export_dir / filename
            
            with open(export_path, 'w', newline='', encoding='utf-8-sig') as f:
                writer = csv.writer(f)
                writer.writerow(["統計項目", "數值"])
                writer.writerow(["總任務數", stats['total']])
                writer.writerow(["已完成", stats['completed']])
                writer.writerow(["進行中", stats['in_progress']])
                writer.writerow(["待處理", stats['pending']])
                writer.writerow(["已逾期", stats['overdue']])
                writer.writerow(["完成率", f"{stats['completion_rate']:.1%}"])
                writer.writerow(["高優先級", stats['high_priority']])
                writer.writerow(["中優先級", stats['medium_priority']])
                writer.writerow(["低優先級", stats['low_priority']])
            
            print(f"✓ 統計報表已匯出至: {export_path}")
            return True
        
        except IOError as e:
            print(f"❌ 統計報表匯出失敗：{e}")
            return False


# 測試 CSV 匯出與統計
print("\n=== CSV 匯出與統計測試 ===")

manager = TodoManager("data")
manager.load_from_file()

# 顯示統計
print("\n1. 測試統計報表...")
manager.display_statistics()

# 匯出 CSV
print("\n2. 測試 CSV 匯出...")
manager.export_to_csv("tasks_export.csv")

# 匯出統計報表
print("\n3. 測試統計報表匯出...")
manager.export_statistics_csv("statistics.csv")

## Part VII: TodoApp 類別 - 命令列介面

### 設計重點

1. **清晰選單**: 結構化選單顯示
2. **輸入驗證**: 完整的使用者輸入驗證
3. **錯誤處理**: 友善的錯誤訊息
4. **迴圈設計**: 持續執行直到使用者選擇退出

In [ ]:
class TodoApp:
    """待辦事項應用程式 - 命令列介面
    
    Attributes:
        manager (TodoManager): 任務管理器
    """
    
    def __init__(self, data_dir: str = "data"):
        """初始化應用程式
        
        Args:
            data_dir: 資料目錄路徑
        """
        self.manager = TodoManager(data_dir)
        self.manager.load_from_file()
        print("\n✓ 待辦事項管理系統啟動成功")
    
    def show_menu(self):
        """顯示主選單"""
        print("\n" + "="*60)
        print("📝 待辦事項管理系統 | Todo Manager v1.0")
        print("="*60)
        print("\n【任務管理】")
        print("  1. 新增任務")
        print("  2. 列出所有任務")
        print("  3. 更新任務")
        print("  4. 刪除任務")
        print("  5. 標記完成")
        print("\n【查詢功能】")
        print("  6. 搜尋任務")
        print("  7. 過濾任務（狀態/優先級）")
        print("  8. 逾期檢查")
        print("\n【報表功能】")
        print("  9. 查看統計")
        print(" 10. 匯出 CSV")
        print("\n【其他】")
        print("  0. 退出")
        print("="*60)
    
    def handle_add_task(self):
        """處理新增任務"""
        print("\n" + "-"*60)
        print("新增任務")
        print("-"*60)
        
        try:
            # 取得使用者輸入
            title = input("📌 標題（必填）: ").strip()
            if not title:
                print("❌ 標題不能為空")
                return
            
            description = input("📄 描述（選填）: ").strip()
            
            priority = input("⭐ 優先級 (High/Medium/Low,預設 Medium): ").strip() or "Medium"
            
            due_date_input = input("📅 截止日期 (YYYY-MM-DD,選填): ").strip()
            due_date = due_date_input if due_date_input else None
            
            tags_input = input("🏷️  標籤（用逗號分隔,選填）: ").strip()
            tags = [tag.strip() for tag in tags_input.split(',')] if tags_input else None
            
            # 新增任務
            task = self.manager.add_task(title, description, priority, due_date, tags)
            
            if task:
                print(f"\n✓ 任務新增成功！")
                print(f"  {task}")
        
        except Exception as e:
            print(f"❌ 新增任務時發生錯誤：{e}")
    
    def handle_list_tasks(self):
        """處理列出任務"""
        print("\n" + "-"*60)
        print("列出任務")
        print("-"*60)
        
        filter_by = input("\n過濾條件（Pending/Completed/High/Medium/Low/Overdue/Upcoming,留空顯示全部）: ").strip()
        filter_by = filter_by if filter_by else None
        
        sort_by = input("排序依據（created/due_date/priority,預設 created）: ").strip() or "created"
        
        self.manager.list_tasks(filter_by, sort_by)
    
    def handle_update_task(self):
        """處理更新任務"""
        print("\n" + "-"*60)
        print("更新任務")
        print("-"*60)
        
        # 先列出所有任務
        self.manager.list_tasks()
        
        task_id = input("\n請輸入任務 ID（前 8 碼）: ").strip()
        if not task_id:
            print("❌ 請輸入任務 ID")
            return
        
        # 取得更新欄位
        print("\n請輸入要更新的欄位（留空表示不更新）：")
        updates = {}
        
        title = input("新標題: ").strip()
        if title:
            updates['title'] = title
        
        description = input("新描述: ").strip()
        if description:
            updates['description'] = description
        
        priority = input("新優先級 (High/Medium/Low): ").strip()
        if priority:
            updates['priority'] = priority
        
        status = input("新狀態 (Pending/InProgress/Completed): ").strip()
        if status:
            updates['status'] = status
        
        due_date = input("新截止日期 (YYYY-MM-DD): ").strip()
        if due_date:
            updates['due_date'] = due_date
        
        if updates:
            self.manager.update_task(task_id, **updates)
        else:
            print("⚠️  沒有任何欄位需要更新")
    
    def handle_delete_task(self):
        """處理刪除任務"""
        print("\n" + "-"*60)
        print("刪除任務")
        print("-"*60)
        
        # 先列出所有任務
        self.manager.list_tasks()
        
        task_id = input("\n請輸入要刪除的任務 ID（前 8 碼）: ").strip()
        if task_id:
            self.manager.delete_task(task_id)
    
    def handle_mark_completed(self):
        """處理標記完成"""
        print("\n" + "-"*60)
        print("標記完成")
        print("-"*60)
        
        # 列出未完成任務
        self.manager.list_tasks(filter_by="Pending")
        
        task_id = input("\n請輸入要標記完成的任務 ID（前 8 碼）: ").strip()
        if task_id:
            self.manager.mark_completed(task_id)
    
    def handle_search(self):
        """處理搜尋任務"""
        print("\n" + "-"*60)
        print("搜尋任務")
        print("-"*60)
        
        keyword = input("\n請輸入搜尋關鍵字: ").strip()
        if keyword:
            self.manager.search_tasks(keyword)
    
    def handle_filter(self):
        """處理過濾任務"""
        print("\n" + "-"*60)
        print("過濾任務")
        print("-"*60)
        print("\n過濾選項：")
        print("  1. 狀態（Pending/InProgress/Completed）")
        print("  2. 優先級（High/Medium/Low）")
        print("  3. 標籤")
        
        choice = input("\n請選擇過濾類型 (1-3): ").strip()
        
        if choice == "1":
            status = input("請輸入狀態: ").strip()
            tasks = self.manager.filter_by_status(status)
            for task in tasks:
                print(f"  - {task}")
        elif choice == "2":
            priority = input("請輸入優先級: ").strip()
            tasks = self.manager.filter_by_priority(priority)
            for task in tasks:
                print(f"  - {task}")
        elif choice == "3":
            # 先顯示所有標籤
            all_tags = self.manager.get_all_tags()
            print(f"\n可用標籤: {', '.join(all_tags)}")
            tag = input("請輸入標籤: ").strip()
            tasks = self.manager.filter_by_tag(tag)
            for task in tasks:
                print(f"  - {task}")
    
    def handle_overdue_check(self):
        """處理逾期檢查"""
        print("\n" + "-"*60)
        print("逾期檢查")
        print("-"*60)
        
        self.manager.get_overdue_tasks()
        self.manager.get_upcoming_tasks(days=7)
    
    def handle_statistics(self):
        """處理查看統計"""
        self.manager.display_statistics()
    
    def handle_export(self):
        """處理匯出 CSV"""
        print("\n" + "-"*60)
        print("匯出 CSV")
        print("-"*60)
        print("\n匯出選項：")
        print("  1. 匯出所有任務")
        print("  2. 匯出統計報表")
        
        choice = input("\n請選擇 (1-2): ").strip()
        
        if choice == "1":
            filename = input("檔案名稱（預設 tasks_export.csv）: ").strip() or "tasks_export.csv"
            self.manager.export_to_csv(filename)
        elif choice == "2":
            filename = input("檔案名稱（預設 statistics.csv）: ").strip() or "statistics.csv"
            self.manager.export_statistics_csv(filename)
    
    def run(self):
        """執行主程式迴圈"""
        while True:
            try:
                self.show_menu()
                choice = input("\n請選擇功能 (0-10): ").strip()
                
                if choice == "0":
                    print("\n感謝使用！再見！👋")
                    break
                elif choice == "1":
                    self.handle_add_task()
                elif choice == "2":
                    self.handle_list_tasks()
                elif choice == "3":
                    self.handle_update_task()
                elif choice == "4":
                    self.handle_delete_task()
                elif choice == "5":
                    self.handle_mark_completed()
                elif choice == "6":
                    self.handle_search()
                elif choice == "7":
                    self.handle_filter()
                elif choice == "8":
                    self.handle_overdue_check()
                elif choice == "9":
                    self.handle_statistics()
                elif choice == "10":
                    self.handle_export()
                else:
                    print("❌ 無效的選項,請重新選擇")
                
                # 等待使用者按 Enter 繼續
                input("\n按 Enter 繼續...")
            
            except KeyboardInterrupt:
                print("\n\n程式已中斷")
                break
            except Exception as e:
                print(f"\n❌ 發生錯誤：{e}")
                input("按 Enter 繼續...")


print("\n✓ TodoApp 類別定義完成")
print("\n提示：執行下一個 Cell 啟動應用程式")

## Part VIII: 完整測試與示範

### 測試案例說明

以下測試案例展示系統的完整功能：

1. **基本 CRUD**: 新增、查看、更新、刪除任務
2. **搜尋過濾**: 關鍵字搜尋、條件過濾
3. **逾期管理**: 逾期檢查、即將到期提醒
4. **統計報表**: 任務統計、CSV 匯出
5. **檔案持久化**: 資料儲存、備份、恢復

In [ ]:
"""完整功能測試與示範"""

print("\n" + "="*80)
print("🧪 待辦事項管理系統 - 完整功能測試")
print("="*80)

# 建立測試管理器
test_manager = TodoManager("data")
test_manager.load_from_file()

print("\n【測試 1】新增測試任務...")
print("-" * 80)

# 新增多個測試任務
task1 = test_manager.add_task(
    "完成 Python 專案報告",
    "實作待辦事項管理系統,包含 JSON、CSV、pathlib",
    "High",
    "2025-10-20",
    ["study", "programming", "project"]
)

task2 = test_manager.add_task(
    "準備資料結構期中考",
    "複習第 1-5 章,練習題目",
    "High",
    "2025-10-18",
    ["study", "exam"]
)

task3 = test_manager.add_task(
    "買菜",
    "購買晚餐食材：蔬菜、肉類、水果",
    "Low",
    tags=["life", "shopping"]
)

task4 = test_manager.add_task(
    "寫部落格文章",
    "主題：Python 檔案處理完全指南",
    "Medium",
    "2025-10-25",
    ["writing", "programming"]
)

task5 = test_manager.add_task(
    "修復專案 Bug",
    "修復登入功能的認證錯誤",
    "High",
    "2025-10-12",
    ["programming", "urgent"]
)

print("\n【測試 2】列出所有任務...")
print("-" * 80)
test_manager.list_tasks(sort_by="priority")

print("\n【測試 3】搜尋功能測試...")
print("-" * 80)
test_manager.search_tasks("Python")

print("\n【測試 4】過濾功能測試 - 高優先級任務...")
print("-" * 80)
test_manager.list_tasks(filter_by="High")

print("\n【測試 5】標籤功能測試...")
print("-" * 80)
all_tags = test_manager.get_all_tags()
print(f"所有標籤: {', '.join(all_tags)}")
tag_stats = test_manager.get_tag_statistics()
print("\n標籤統計:")
for tag, count in tag_stats.items():
    print(f"  #{tag}: {count} 次")

print("\n【測試 6】更新任務測試...")
print("-" * 80)
if task2:
    test_manager.mark_in_progress(task2.id[:8])
if task3:
    test_manager.mark_completed(task3.id[:8])

print("\n【測試 7】逾期檢查測試...")
print("-" * 80)
test_manager.get_overdue_tasks()
test_manager.get_upcoming_tasks(days=10)

print("\n【測試 8】統計報表測試...")
print("-" * 80)
test_manager.display_statistics()

print("\n【測試 9】CSV 匯出測試...")
print("-" * 80)
test_manager.export_to_csv("test_tasks.csv")
test_manager.export_statistics_csv("test_statistics.csv")

print("\n【測試 10】檔案持久化測試...")
print("-" * 80)
print("1. 儲存資料...")
test_manager.save_to_file()

print("\n2. 建立新管理器並讀取資料...")
new_manager = TodoManager("data")
new_manager.load_from_file()
print(f"✓ 成功載入 {len(new_manager.tasks)} 個任務")

print("\n3. 驗證資料完整性...")
if len(test_manager.tasks) == len(new_manager.tasks):
    print("✓ 資料完整性驗證通過")
else:
    print("❌ 資料完整性驗證失敗")

print("\n" + "="*80)
print("✓ 所有測試完成！")
print("="*80)

print("\n📁 檔案清單：")
data_dir = Path("data")
if data_dir.exists():
    for file in data_dir.rglob("*"):
        if file.is_file():
            size = file.stat().st_size
            print(f"  - {file.relative_to(data_dir)}: {size:,} bytes")

## Part IX: 啟動應用程式

**注意**: 在 Jupyter Notebook 中執行互動式應用程式可能會遇到限制。
建議將程式碼儲存為 `.py` 檔案後在終端機執行。

### 使用方式

```python
# 方法 1: 直接執行（Jupyter 中）
app = TodoApp("data")
# app.run()  # 取消註解以啟動（建議在終端機執行）

# 方法 2: 儲存為 .py 檔案後執行
# 1. 將所有 Cell 匯出為 .py
# 2. 在終端機執行: python todo_app.py
```

In [ ]:
"""啟動應用程式（互動式介面）"""

if __name__ == "__main__":
    print("\n" + "="*80)
    print("待辦事項管理系統啟動")
    print("="*80)
    print("\n⚠️  注意：在 Jupyter Notebook 中執行互動式程式可能有限制")
    print("建議：將此 notebook 匯出為 .py 檔案後在終端機執行")
    print("\n如要啟動,請取消註解下方程式碼：\n")
    
    # 取消註解以啟動應用程式
    # app = TodoApp("data")
    # app.run()
    
    print("\n示範模式：顯示主選單")
    demo_app = TodoApp("data")
    demo_app.show_menu()
    
    print("\n✓ 如需使用完整功能,請：")
    print("  1. 匯出為 Python 檔案：File → Download as → Python (.py)")
    print("  2. 在終端機執行：python solution.py")

## 📚 學習總結

### 🎯 完成度檢核

| 項目 | 配分 | 完成度 |
|:-----|:----:|:------:|
| **基本需求** | 70 | ✅ 100% |
| Task 資料結構 | 10 | ✅ |
| 檔案持久化 | 15 | ✅ |
| CRUD 操作 | 25 | ✅ |
| 搜尋與過濾 | 10 | ✅ |
| CLI 介面 | 10 | ✅ |
| **進階需求** | 30 | ✅ 100% |
| 截止日期管理 | 8 | ✅ |
| CSV 匯出 | 8 | ✅ |
| 統計報表 | 5 | ✅ |
| 標籤系統 | 6 | ✅ |
| 逾期檢查 | 3 | ✅ |
| **總計** | 100 | **100%** |

---

### 💡 核心技術重點

#### 1. JSON 序列化（Ch24）
```python
# to_dict: 物件 → 字典
def to_dict(self) -> Dict:
    return {
        "created_date": self.created_date.isoformat()  # datetime → ISO 字串
    }

# from_dict: 字典 → 物件
@classmethod
def from_dict(cls, data: Dict) -> 'Task':
    return cls(
        created_date=data["created_date"]  # ISO 字串 → datetime
    )
```

#### 2. pathlib 路徑管理（Ch26）
```python
# 跨平台路徑處理
self.data_dir = Path("data")
self.data_file = self.data_dir / "todos.json"  # 使用 / 運算子
self.data_dir.mkdir(exist_ok=True)  # 建立目錄
```

#### 3. CSV 匯出（Ch25）
```python
# 使用 DictWriter 匯出
with open(export_path, 'w', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({...})
```

#### 4. 錯誤處理與備份（Ch20-21）
```python
try:
    # 主要操作
except json.JSONDecodeError:
    # 從備份恢復
    self._restore_from_backup()
```

---

### 🚀 擴展建議

1. **子任務系統**: 支援任務階層（父任務 → 子任務）
2. **提醒通知**: 整合系統通知功能
3. **資料加密**: 敏感資料加密儲存
4. **雲端同步**: 整合 Google Drive / Dropbox API
5. **GUI 介面**: 使用 tkinter 或 PyQt 建立圖形介面
6. **Web API**: 使用 Flask/FastAPI 提供 RESTful API
7. **資料庫版本**: 改用 SQLite 儲存資料

---

### 📖 延伸閱讀

- **Python 官方文件**:
  - json: https://docs.python.org/3/library/json.html
  - csv: https://docs.python.org/3/library/csv.html
  - pathlib: https://docs.python.org/3/library/pathlib.html
  - datetime: https://docs.python.org/3/library/datetime.html

- **相關專案參考**:
  - Todoist: https://todoist.com
  - Microsoft To Do: https://todo.microsoft.com
  - Taskwarrior: https://taskwarrior.org

---

**專案完成！恭喜您掌握了檔案處理與資料持久化的核心技術！** 🎉

---

**參考解答版本**: v1.0  
**最後更新**: 2025-10-08  
**維護者**: iSpan Python 教學團隊